In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np
from functools import reduce
from datetime import datetime

In [2]:
locations = (
    (33, 4),
    (468, 3),
    (21, 3),
    (21, 3)
)
keypoints_len = reduce(lambda r, loc: r + loc[0] * loc[1], locations, 0)
keypoints_len

1662

In [3]:
def extract_keypoints(results):
  pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
  face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
  lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
  rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
  return np.concatenate([pose, face, lh, rh])

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def draw_styled_landmarks(image, results):
    # # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [10]:
VIDEO_PATH = "s2s_vids"
for word in os.listdir(VIDEO_PATH):
    print(f'{datetime.now().time()} | Start processing {word}')
    word_path = os.path.join(VIDEO_PATH, word)

    for i, name in enumerate(os.listdir(word_path)):

        print(i, name[:-4])
      
        # word_video_path = os.path.join(word_path, name)

        # cap = cv2.VideoCapture(word_video_path)
        # frame_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # track = np.empty([frame_len, keypoints_len])

        # for i in range(frame_len):
            # ret, frame = cap.read()

            # landmarks = holistic.process(
            #     cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # )

            # track[i] = extract_keypoints(landmarks)
        
        # save_path = os.path.join('tracks_binary', word)
        # os.makedirs(save_path, exist_ok=True)
        # track.astype('float32').tofile(f'tracks_binary/{word}/{name}')

# cap.release()

23:14:19.492042 | Start processing buy
0 buy1
1 buy10
2 buy11
3 buy12
4 buy13
5 buy14
6 buy15
7 buy2
8 buy4
9 buy5
10 buy6
11 buy7
12 buy8
13 buy9
23:14:19.495056 | Start processing do
0 do2
1 do3
2 do4
3 do5
4 do6
5 do7
6 do8
7 README
23:14:19.497042 | Start processing eat
0 eat1
1 eat10
2 eat11
3 eat12
4 eat13
5 eat14
6 eat15
7 eat16
8 eat18
9 eat19
10 eat2
11 eat20
12 eat21
13 eat23
14 eat24
15 eat25
16 eat26
17 eat27
18 eat28
19 eat29
20 eat30
21 eat33
22 eat34
23 eat35
24 eat36
25 eat37
26 eat38
27 eat39
28 eat40
29 eat41
30 eat42
31 eat43
32 eat44
33 eat46
34 eat47
35 eat5
36 eat6
37 eat7
38 eat8
39 eat9
23:14:19.505045 | Start processing fine
0 fine1
1 fine10
2 fine11
3 fine12
4 fine13
5 fine14
6 fine15
7 fine16
8 fine17
9 fine18
10 fine19
11 fine2
12 fine20
13 fine21
14 fine22
15 fine23
16 fine24
17 fine25
18 fine26
19 fine27
20 fine28
21 fine29
22 fine3
23 fine4
24 fine5
25 fine6
26 fine7
27 fine8
28 fine9
23:14:19.510056 | Start processing go
0 go1
1 go10
2 go11
3 go12
4 go13

In [7]:
with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for word in os.listdir(VIDEO_PATH):
        print(f'{datetime.now().time()} | Start processing {word}')
        word_path = os.path.join(VIDEO_PATH, word)

        for i, name in enumerate(os.listdir(word_path)):
            word_video_path = os.path.join(word_path, name)

            cap = cv2.VideoCapture(word_video_path)
            frame_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            track = np.empty([frame_len, keypoints_len])

            for i in range(frame_len):
                ret, frame = cap.read()

                landmarks = holistic.process(
                    cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                )

                track[i] = extract_keypoints(landmarks)
            
            save_path = os.path.join('tracks_binary', word)
            os.makedirs(save_path, exist_ok=True)
            track.astype('float32').tofile(f'tracks_binary/{word}/{name}')

    cap.release()

NameError: name 'VIDEO_PATH' is not defined

In [20]:
for word in words[8:]:
  print(word)

thanks
yes


In [21]:
words = os.listdir('videos')
num_vids = 20
frame_len = 64

cap = cv2.VideoCapture(0)

with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  for word in words:
    
    for video in range(num_vids):
      track = np.empty([frame_len, keypoints_len])

      for frame_num in range(frame_len):
        ret, frame = cap.read()

        image, landmarks = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, landmarks)
        
        if frame_num == 0: 
          cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
          cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(word, video), (15,12), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
          # Show to screen
          cv2.imshow('OpenCV Feed', image)
          cv2.waitKey(500)
        else: 
          cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(word, video), (15,12), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
          # Show to screen
          cv2.imshow('OpenCV Feed', image)
        

        track[frame_num] = extract_keypoints(landmarks)
      
        save_path = os.path.join('tracks_binary_manual', word)
        os.makedirs(save_path, exist_ok=True)
        track.astype('float32').tofile(f'tracks_binary_manual/{word}/{video}')
      
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
          break

  cap.release()
  cv2.destroyAllWindows()
  